In [ ]:
# Reading the Libraries
import pandas as pd
import numpy as np

# Reading the Data
data = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
data.head(20)

In [ ]:
data.shape

Let's have a look at the health of the data itself. This function `.info()` in the pandas library is very helpful to understand the basic properties of the data itself. If there is any missing values in the dataset can be known right from here so that they can be taken care of before fitting into a model for training and testing. 


In [ ]:
data.info()

**There are 4 things that take my attention**

1. There is an **`id`** that cannot be used for classificaiton 
2. **`Diagnosis`** column in the data is our class label
3. **`Unnamed: 32`** feature includes **`NaN`** so we do not need it.
4. I do not have any idea about other feature names actually.

Therefore, drop these unnecessary features. However do not forget this is not a feature selection. This is like a browse a pub, we do not choose our drink yet !!!

In [ ]:
# feature names as a list
# .columns gives columns names in data 
col = data.columns       
print(col)


data.drop(['Unnamed: 32',"id"], axis=1, inplace=True)
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]
y_data = data.diagnosis.values
x_data = data.drop(['diagnosis'], axis=1)
x_data

In [ ]:
y_data

Okey, now we have features but **`what does they mean`** or actually **`how much do we need to know about these features`**
The answer is that we do not need to know meaning of these features however in order to imagine in our mind we should know something like variance, standart deviation, number of sample (count) or max min values.
These type of information helps to understand about what is going on data. For example , the question is appeared in my mind the **`area_mean`** feature's max value is 2500 and **`smoothness_mean`** features' max 0.16340. Therefore **do we need standirdization or normalization before visualization, feature selection, feature extraction or classificaiton?** The answer is yes and no not surprising ha :) Anyway lets go step by step and start with visualization. 

## Normalization
Normalization refers to rescaling real-valued numeric attributes into a 0 to 1 range. Data normalization is used in machine learning to make model training less sensitive to the scale of features.

You can either implement the conversion process with basic python or use `MinMaxScaler()` function from the `sklearn` library. 

In [ ]:
# Using transformer from sklearn library
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
output = scalar.fit_transform(x_data)

# Manual Implementation of the normalization process
X_data = (x_data -np.min(x_data))/ (np.max(x_data)-np.min(x_data)).values

In [ ]:
X_data

## The Basics: Logistic Regression and Regularization

Logistic Regression is one of the most common machine learning algorithms used for classification. It a statistical model that uses a logistic function to model a binary dependent variable. In essence, it predicts the probability of an observation belonging to a certain class or label. For instance, is this a cat photo or a dog photo?

Ordinary Least Squares linear regression is powerful and versatile right out of the box, but there are certain circumstances where it fails. 
1. it is, expressly, a ‘regression’ framework, which makes it hard to apply as a classifier.
2. unlike, say, a decision tree, linear regression models don’t perform their own implicit feature selection, meaning they are prone to overfit if too many features are included. 

Luckily, there are some extensions to the linear model that allow us to overcome these issues. Logistic regression turns the linear regression framework into a classifier and various types of **`regularization`** of which the `Ridge` and `Lasso` methods are most common, help avoid overfit in feature rich instances.

### **Hypothesis:** 
We want our model to predict the probability of an observation belonging to a certain class or label. As such, we want a hypothesis $h$ that satisfies the following condition $0 <= h(x) <= 1$ , where $x$ is an observation.

We define $h(x) = g(w^T * x)$ , where $g$ is a sigmoid function and $w$ are the trainable parameters or `weights`. As such, we have:
$$h(x) = \frac{1}{1+e^{-w^Tx}}$$

### The cost for an observation: 
Now that we can predict the probability for an observation, we want the result to have the minimum error. If the class label is $y$, the cost (error) associated with an observation $x$ is given by:

![](https://miro.medium.com/max/525/1*vSGnYVz6I7sAObKuxuFAoQ.gif)

### Cost Function: 
Thus, the total cost for all the $m$ observations in a dataset is:
![](https://miro.medium.com/max/368/0*vZnp94vCoN0vMDAj)

We can rewrite the cost function J as:
![](https://miro.medium.com/max/691/0*o57ug0iMGDJVI1qo)

The objective of logistic regression is to find params `w` so that `J` is minimum. How can we do that?? We will use the gradient descent algorithm to update each of the weights gradually to minimize the cost `J`. 

We will update each of the params wᵢ using the following template:
![](https://miro.medium.com/max/875/0*Q6ssvXABrvHUZrfy)
![](https://miro.medium.com/max/496/0*7uVvuW-ZGauNWH_V)

The above step will help us find a set of params wᵢ, which will then help us to come up with $h(x)$ to solve our binary classification task.
But there is also an undesirable outcome associated with the above gradient descent steps. In an attempt to find the best $h(x)$, the following things happen:

**CASE I: For class label = 0**: $h(x)$ will try to produce results as close 0 as possible. As such, $w^T.x$ will be as small as possible
=> Wi will tend to -infinity

**CASE II: For class label = 1**: $h(x)$ will try to produce results as close 1 as possible. As such, $w^T.x$ will be as large as possible
=> Wi will tend to +infinity


## Regularization:
Regularization is a technique to solve the problem of overfitting in a machine learning algorithm by penalizing the cost function. It does so by using an additional penalty term in the cost function.
There are two types of regularization techniques:
1. Lasso or L1 Regularization
2. Ridge or L2 Regularization (We will implement here)
So, how can L2 Regularization help to prevent overfitting? Let’s first look at our new cost function:

![](https://miro.medium.com/max/628/0*Nc_ocecF0dHpUutK)


The regularization term will heavily penalize large $w_i$. The effect will be less on smaller $w_i$’s. As such, the growth of $w$ is controlled. The $h(x)$ we obtain with these controlled params $w$ will be more generalizable.

**NOTE:** $λ$ is a hyper-parameter value. We have to find it using cross-validation. 
* Larger value $λ$ of will make $w_i$ shrink closer to $0$, which might lead to underfitting. 
* $λ = 0$, will have no regulariztion effect. 

When choosing $λ$, we have to take proper care of bias vs variance trade-off.


In [ ]:
class LogisticRegression(object):
    """
    Logistic Regression Classifier
    Parameters
    ----------
    learning_rate : int or float, default=0.1
        The tuning parameter for the optimization algorithm (here, Gradient Descent) 
        that determines the step size at each iteration while moving toward a minimum 
        of the cost function.
    max_iter : int, default=100
        Maximum number of iterations taken for the optimization algorithm to converge
    
    penalty : None or 'l2', default='l2'.
        Option to perform L2 regularization.
    C : float, default=0.1
        Inverse of regularization strength; must be a positive float. 
        Smaller values specify stronger regularization. 
    tolerance : float, optional, default=1e-4
        Value indicating the weight change between epochs in which
        gradient descent should terminated. 
    """

    def __init__(self, learning_rate=0.1, max_iter=100, regularization='l2', lambda_ = 10 , tolerance = 1e-4):
        self.learning_rate  = learning_rate
        self.max_iter       = max_iter
        self.regularization = regularization
        self.lambda_        = lambda_
        self.tolerance      = tolerance
        self.loss_log       = []
    
    def fit(self, X, y, verbose = False):
        """
        Fit the model according to the given training data.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training vector, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like of shape (n_samples,)
            Target vector relative to X.
        Returns
        -------
        self : object
        """
        self.theta = np.random.rand(X.shape[1] + 1)
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
        self.loss_log = []

        for iteration in range(self.max_iter):
            Z = np.matmul(X,  self.theta)
            y_hat = self.__sigmoid(Z)
            
            errors = y_hat - y

            N = X.shape[1] 
            
            cost = (-1.0/N) * np.sum( y*np.log(y_hat) + (1.0 - y)*np.log(1.0-y_hat))
            self.loss_log.append(cost)
            
            if verbose:
                print(f'Iteration {iteration} Loss: {cost}')

            if self.regularization is not None:
                delta_grad = (1./N) *(np.matmul(errors.T, X)+ self.lambda_ * self.theta)
            else:
                delta_grad = (1./N) *(np.matmul(errors.T, X))
                
            self.theta -= self.learning_rate * delta_grad

#             if np.all(abs(delta_grad) >= self.tolerance):
#                 self.theta -= self.learning_rate * delta_grad
#             else:
#                 break
                
        return self

    def predict_proba(self, X):
        """
        Probability estimates for samples in X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Vector to be scored, where `n_samples` is the number of samples and
            `n_features` is the number of features.
        Returns
        -------
        probs : array-like of shape (n_samples,)
            Returns the probability of each sample.
        """
        return self.__sigmoid((X @ self.theta[1:]) + self.theta[0])
    
    def predict(self, X):
        """
        Predict class labels for samples in X.
        Parameters
        ----------
        X : array_like or sparse matrix, shape (n_samples, n_features)
            Samples.
        Returns
        -------
        labels : array, shape [n_samples]
            Predicted class label per sample.
        """
        return np.round(self.predict_proba(X))
        
    def __sigmoid(self, z):
        """
        The sigmoid function.
        Parameters
        ------------
        z : float
            linear combinations of weights and sample features
            z = w_0 + w_1*x_1 + ... + w_n*x_n
        Returns
        ---------
        Value of logistic function at z
        """
        return (1.0 / (1.0 + np.exp(-z)))

    def get_params(self):
        """
        Get method for models coeffients and intercept.
        Returns
        -------
        params : dict
        """
        try:
            params = dict()
            params['intercept'] = self.theta[0]
            params['coef'] = self.theta[1:]
            return params
        except:
            raise Exception('Fit the model first!')

### Train and Test Data Validation
Let us split the whole data into two portion. We take 80% data in the train set and then put rest of the data into the test set to check the performance of the trained model. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)

# Train and Test Data Summary
import plotly.graph_objects as go
split = ['Train','Test']

fig = go.Figure()
fig.add_trace(go.Bar(x=split, y=[np.sum(y_train), np.sum(y_test)],#                base=[-500,-600],
                    marker_color='crimson',
                    name='Malignant'))
fig.add_trace(go.Bar(x=split, 
                     y=[len(y_train)- np.sum(y_train), len(y_test) - np.sum(y_test)],
                    base=0,
                    marker_color='lightgreen',
                    name='Benign'                ))
fig.update_layout(width = 800, height = 400)
fig.update_layout(title = 'Count of Samples in Train and Test Split', title_x = 0.5, xaxis_title = "Category", yaxis_title = 'Sample Count')
fig.show()

## Training Logistic Regression
Now we train the logistic regression with the training data for a maximum interation of 200. Other parameters are kept default. Feel free to fiddle around the other parameters to understand more of them. 

In [ ]:
MAX_ITER = 400
LR_RATE = 1e-2

clf_no_reg = LogisticRegression(max_iter = MAX_ITER, learning_rate= LR_RATE,  lambda_ = 200, regularization= None)
clf_no_reg.fit(X_train, y_train, verbose = False)

clf_reg_10 = LogisticRegression(max_iter = MAX_ITER, learning_rate= LR_RATE, lambda_ = 10, regularization= 'l2')
clf_reg_10.fit(X_train, y_train, verbose = False)


clf_reg_20 = LogisticRegression(max_iter = MAX_ITER, learning_rate= LR_RATE, lambda_ = 20, regularization= 'l2')
clf_reg_20.fit(X_train, y_train, verbose = False)

clf_reg_40 = LogisticRegression(max_iter = MAX_ITER, learning_rate= LR_RATE, lambda_ = 40, regularization= 'l2')
clf_reg_40.fit(X_train, y_train, verbose = False)


### Plot Training Loss over Time 

In [ ]:
import plotly.graph_objects as go
import numpy as np

y = clf_no_reg.loss_log 

# fig = go.Figure(data=go.Scatter(x= np.arange(start =1, stop = len(y)), 
#                                 y=y,
#                                 mode = 'lines+markers'))


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(start =1, stop = len(y)), y=clf_no_reg.loss_log ,
                    mode='lines+markers',
                    name='No Regularization'))

fig.add_trace(go.Scatter(x=np.arange(start =1, stop = len(y)), y=clf_reg_10.loss_log ,
                    mode='lines+markers',
                    name='Regulrization W=10'))

fig.add_trace(go.Scatter(x=np.arange(start =1, stop = len(y)), y=clf_reg_20.loss_log ,
                    mode='lines+markers',
                    name='Regulrization W=20'))

fig.add_trace(go.Scatter(x=np.arange(start =1, stop = len(y)), y=clf_reg_40.loss_log ,
                    mode='lines+markers',
                    name='Regulrization W=40'))


fig.update_layout(title = "Error Plot over Iterations", title_x = 0.5,
                  xaxis_title = 'Iteration',
                  yaxis_title = 'Log Loss',
                  width = 800,
                  height = 500)
fig.show()

## Accuracy Calculation

In [ ]:
from sklearn import metrics
print(f'Accuracy {metrics.accuracy_score(y_test, clf_no_reg.predict(X_test))*100}%')
print(f'Accuracy {metrics.accuracy_score(y_test, clf_reg_10.predict(X_test))*100}%')
print(f'Accuracy {metrics.accuracy_score(y_test, clf_reg_20.predict(X_test))*100}%')
print(f'Accuracy {metrics.accuracy_score(y_test, clf_reg_40.predict(X_test))*100}%')

## Plot Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, clf_no_reg.predict(X_test)))
sns.heatmap(confusion_matrix(y_test, clf_no_reg.predict(X_test)), annot=True)